In [305]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [306]:
import torch
import sophius.utils as utils
import sophius.dataload as dload
from sophius.modelgen import ConvModelGenerator
from sophius.train import train_express_gpu
import torchvision.datasets as dset
import torchvision.transforms as T

In [307]:
VAL_SIZE = 1024

cifar10 = dset.CIFAR10('../data/CIFAR10', train=True, download=True,
                           transform=T.ToTensor())
cifar_gpu = dload.cifar_to_gpu(cifar10)

Files already downloaded and verified


In [308]:
model_gen = ConvModelGenerator((3, 32, 32), 10, conv_num=6, lin_num=3)
model_tmpl = model_gen.generate_model_tmpl()
print(model_tmpl)
fixed_model_gpu = model_tmpl.instantiate_model().type(torch.cuda.FloatTensor)

Conv2d       (96, 32, 32)   (5, 5)   (1, 1)  
PReLU        (96, 32, 32)  
AvgPool2d    (96, 17, 17)   (2, 2)   (2, 2)  
Conv2d       (192, 6, 6)    (6, 6)   (2, 2)  
PReLU        (192, 6, 6)   
AvgPool2d    (192, 6, 6)    (3, 3)   (1, 1)  
MaxPool2d    (192, 6, 6)    (3, 3)   (1, 1)  
Conv2d       (64, 1, 1)     (6, 6)   (3, 3)  
ReLU         (64, 1, 1)    
Conv2d       (32, 1, 1)     (2, 2)   (3, 3)  
Flatten      32            
Linear       512           
ReLU         512           
Linear       10            



In [309]:
fixed_model_gpu

Sequential(
  (0): Conv2d(3, 96, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (1): PReLU(num_parameters=96)
  (2): AvgPool2d(kernel_size=(2, 2), stride=(2, 2), padding=(1, 1))
  (3): Conv2d(96, 192, kernel_size=(6, 6), stride=(2, 2))
  (4): PReLU(num_parameters=1)
  (5): AvgPool2d(kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): MaxPool2d(kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), dilation=(1, 1), ceil_mode=False)
  (7): Conv2d(192, 64, kernel_size=(6, 6), stride=(3, 3))
  (8): ReLU()
  (9): Conv2d(64, 32, kernel_size=(2, 2), stride=(3, 3), padding=(1, 1))
  (10): Flatten()
  (11): Linear(in_features=32, out_features=512, bias=True)
  (12): ReLU()
  (13): Linear(in_features=512, out_features=10, bias=True)
)

In [310]:
loader_gpu = dload.get_loader_gpu(cifar_gpu, val_size=VAL_SIZE, batch_size=1024)

# express train
t, val_acc, train_acc = train_express_gpu(model = fixed_model_gpu,
                                          train = True,
                                          loader = loader_gpu,
                                          milestones = [],
                                          num_epoch = 1,
                                          verbose = True)

Finished in 50.6s 
val_acc: 0.301, train_acc: 0.325


In [314]:
from sophius.templates import *

tmpl_list = [
    LinearTmpl(),
    BatchNorm2dTmpl(),
    ReLUTmpl(),
    LeakyReLUTmpl(),
    PReLUTmpl(),
    DropoutTmpl(),
    Dropout2dTmpl(),
    FlattenTmpl(),
    Conv2dTmpl(),
    MaxPool2dTmpl(),
    AvgPool2dTmpl(),
    GlobalAvgPool2dTmpl(),
]

for tmpl in tmpl_list:
    print(type(tmpl).__name__)
    # print(tmpl.get_learnable_params())

LinearTmpl
BatchNorm2dTmpl
ReLUTmpl
LeakyReLUTmpl
PReLUTmpl
DropoutTmpl
Dropout2dTmpl
FlattenTmpl
Conv2dTmpl
MaxPool2dTmpl
AvgPool2dTmpl
GlobalAvgPool2dTmpl


In [315]:
Conv2dTmpl().config

{'out_channels': 32,
 'kernel_size': (3, 3),
 'stride': (1, 1),
 'padding': True,
 'padding_mode': zeros,
 'dilation': (1, 1),
 'groups': 1,
 'bias': True}

In [316]:
from sklearn.preprocessing import OneHotEncoder

class LayerEncoder:
    def __init__(self, layers, size=32):
        self.layers = layers
        self.layer_names = [x.name for x in layers]
        self.size = size
        
    def encode_layer(self, layer):
        # One-hot encoding for the layer type
        type_encoding = np.zeros(len(self.layer_types), dtype=np.uint32)
        idx = self.layer_types.index(layer.name)
        type_encoding[idx] = 1
        
        param_encoding = []
        
        for param_name, param_data in layer.config_data.items():            
            # skip non-learnable params
            if not layer.config[param_name].learnable:
                continue
                
            value = layer.config[param_name].value
            param_range = param_data.get('range')
            
            # Handle discrete ranges (categorical values)
            if isinstance(param_range, list):
                encoding = np.zeros(len(param_range), dtype=np.uint32)
                if value in param_range:
                    idx = param_range.index(value)
                    encoding[idx] = 1
                else:
                    raise ValueError(f"Value {value} not in the provided range for {param_name}")
                        
            else:
                raise ValueError(f"Unsupported parameter type for {param_name}: {value}")
            
            param_encoding.append(encoding)        
        
        # Concatenate all encodings into a single vector for the layer        
        final_encoding = np.concatenate([type_encoding] + param_encoding)
        final_encoding = np.pad(final_encoding, (0, max(0, self.size - len(final_encoding))))

        
        return final_encoding

# Example usage
layers = [
    LinearTmpl(),
    BatchNorm2dTmpl(),
    ReLUTmpl(),
    LeakyReLUTmpl(),
    PReLUTmpl(),
    DropoutTmpl(),
    Dropout2dTmpl(),
    FlattenTmpl(),
    Conv2dTmpl(),
    MaxPool2dTmpl(),
    AvgPool2dTmpl(),
    GlobalAvgPool2dTmpl()
]

encoder = LayerEncoder(layers)

for layer in layers:
# layer = random.choice(layers)
    layer.gen_rand_config()    
    encoded_layer = encoder.encode_layer(layer)
    
    # print(layer.name, layer.get_learnable_params())
    # print(len(encoded_layer))
    print(encoded_layer)

[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 1 0 0 1 0]
[0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
